# SQL Schema

## Overzicht

Het schema van een database beschrijft de *structuur* daarvan: de namen en types van tabellen en kolommen, en de bijbehorende constraints.
Het schema van een tabel heeft gewoonlijk de vorm van een SQL-opdracht om de betreffende tabel aan te maken.
Het schema is in het DBMS opgeslagen, via SQL-opdrachten kun je dit schema dan opvragen.

Een schema kan *constraints* bevatten: eisen die aan de data in de database gesteld worden,
zoals de uniciteit van de waarden in een kolom, of de "referential integrity" (zie verderop) van foreign keys.
We geven voorbeelden van opdrachten om te demonstreren dat het DBMS opdrachten die deze constraints schenden niet uitvoert: we kunnen er dan zeker van zijn dat de waarden in de database aan deze constraints voldoen.

In [27]:
%LOAD example.db

## Schema

Het *schema* van een database beschrijft de structuur: de tabellen, kolomnamen, types en constraints.
Het schema van een database is in het DBMS (hier: SQLite) opgeslagen, in een aparte database (`sqlite_master`).
Een schema kun je bijvoorbeeld weergeven in de vorm van de "create table" opdrachten.

## Schema opvragen via SQL

### Alle tabellen

Met de volgende SQL-query vraag je (in SQLite) alle tabellen van de huidige database op:

In [3]:
SELECT name FROM sqlite_master WHERE type='table';

name
leden
events
inschrijvingen


In [29]:
SELECT *
FROM sqlite_master
WHERE type = 'table';

type,name,tbl_name,rootpage,sql
table,leden,leden,2,"CREATE TABLE leden( lidnr INTEGER PRIMARY KEY, voornaam VARCHAR(255) NOT NULL, achternaam VARCHAR(255) NOT NULL, email VARCHAR(255) NOT NULL UNIQUE )"
table,events,events,4,"CREATE TABLE events( eventnr INTEGER, datum VARCHAR(10) NOT NULL, beschrijving VARCHAR(255), PRIMARY KEY (eventnr), CONSTRAINT name UNIQUE (datum, beschrijving) )"
table,inschrijvingen,inschrijvingen,6,"CREATE TABLE inschrijvingen( eventnr INTEGER, lidnr INTEGER, maaltijd VARCHAR(255), PRIMARY KEY (lidnr, eventnr), FOREIGN KEY (lidnr) REFERENCES leden (lidnr), FOREIGN KEY (eventnr) REFERENCES events (eventnr) )"


Voor de kolomnamen van de sqlite_master tabel, zie: https://www.techonthenet.com/sqlite/sys_tables/index.php

### Schema van een tabel

Het schema (de structuur) van een tabel vraag je als volgt op in SQL:

In [4]:
SELECT sql FROM sqlite_master WHERE name='leden'

sql
"CREATE TABLE leden( lidnr INTEGER PRIMARY KEY, voornaam VARCHAR(255) NOT NULL, achternaam VARCHAR(255) NOT NULL, email VARCHAR(255) NOT NULL UNIQUE )"


### Opdracht

Vraag op dezelfde manier het schema van de tabel `inschrijvingen` op.

### Opdracht

Vergelijk het database-schema met de onderstaande figuur van het logische model:




## Constraints

Het schema van een database bepaalt welke gegevens in de database opgenomen kunnen worden.
De constraints in het schema geven verdere eisen aan deze gegevens.
Voorbeelden van constraints in de voorbeeld-database:

* de voornaam en de achternaam mogen niet leeg (NULL) zijn;
* de email-waarde van een lid moet uniek zijn;
* de combinatie (datum, beschrijving) van een event moet uniek zijn;
* de verwijzingen (*foreign keys*) in een inschrijving moeten verwijzen naar een bestaand lid en een bestaand event.


### Opdracht

In deze opdracht zie je hoe het DMBS probeert de constraints van het schema probeert te handhaven.

(Zie de eerdere opdrachten in [SQLite - CRUD](sqlite-3.ipynb))

Voer de volgende opdracht uit:

In [6]:
INSERT INTO leden (voornaam, achternaam, email) 
           VALUES ('Donald', 'Duck', 'donald@disney.com');

Error: UNIQUE constraint failed: leden.email

Voer de bovenstaande opdracht *nog een keer* uit.
Welke foutmelding krijg je in dit geval?
Welke constraint betreft dit?

:::{exercise} Meerdere records met dezelfde naam

* maak een opdracht om een tweede lid met de naam "Donald Duck" aan de tabel toe te voegen, met een ander email-adres;
* heeft dat zin in de praktijk?
* voer de opdracht uit; en laat zien dat er inderdaad meerdere leden met die naam zijn.
:::

In [9]:
INSERT INTO leden (voornaam, achternaam, email) 
                  VALUES ('Donald', 'Duck', 'donald@disney.nl');

In [10]:
SELECT * FROM leden;

lidnr,voornaam,achternaam,email
1,Hans,de Boer,hdebo@ziggo.nl
2,Marie,Verkerk,marie@verkerk.nl
3,Janny,Gerards,jgerards@gmail.com
4,Marie,Zandstra,marie123@ziggo.nl
5,Donald,Duck,donald@disney.com
6,Donald,Duck,donaldx@disney.nl
7,Donald,Duck,donald@disney.nl


Ruim je experimentele gegevens op: verwijder alle leden met de naam Donald Duck:

In [11]:
DELETE FROM leden
WHERE voornaam='Donald' AND achternaam='Duck';

en controleer het resultaat:

In [12]:
SELECT * FROM leden;

lidnr,voornaam,achternaam,email
1,Hans,de Boer,hdebo@ziggo.nl
2,Marie,Verkerk,marie@verkerk.nl
3,Janny,Gerards,jgerards@gmail.com
4,Marie,Zandstra,marie123@ziggo.nl


## Toevoegen van een inschrijving

### Referential integrity 

In de volgende voorbeelden proberen we opdrachten uit te voeren die de "referential integrity" van de *foreign keys* dreigen te verstoren.

> Een *foreign key* is een *key* die verwijst naar een rij in een andere (doel)tabel. Voorbeelden hiervan zijn `eventnr` en `lidnr` in de `inschrijvingen`-tabel. We moeten garanderen dat elke rij waar een foreign key naar verwijst inderdaad bestaat in de doel-tabel: dit heet *referential integrity*. Het DBMS moet dit controleren bij het toevoegen van een verwijzing (hier: een *inschrijving*), en bij het verwijderen van rijen waarnaar verwezen kan worden (zoals een `lid` of een `event`). Ook bij het veranderen van een foreign key moet dit gecontroleerd worden.

### Check aan in SQLite?

We controleren eerst of de (SQLite) database juist ingesteld is voor het controleren van de *foreign key constraints*.
In het geval van MySQL staat die controle altijd aan.
Het resultaat van de onderstaande opdracht moet zijn `(1,)`.

In [ ]:
cursor.execute('''PRAGMA foreign_keys;''')
for row in cursor:
    print(row)

In [14]:
INSERT INTO inschrijvingen 
  (eventnr, lidnr, maaltijd)
  VALUES (22, 1, 'maaltijd A');

Error: UNIQUE constraint failed: inschrijvingen.lidnr, inschrijvingen.eventnr

### Toevoegen van inschrijving voor niet-bestaand event

Als eerste proberem we een inschrijving voor een niet-bestaande event toe te voegen.

**Opdracht** Voer de onderstaande cel uit.

* welke foutmelding krijg je?
* verander de `22` in `2`; welke foutmelding krijg je dan?

In [17]:
INSERT INTO inschrijvingen 
  (eventnr, lidnr, maaltijd)
  VALUES (22, 1, 'maaltijd A');

Error: UNIQUE constraint failed: inschrijvingen.lidnr, inschrijvingen.eventnr

## Verwijderen van een event met inschrijvingen

Een volgende poging is om een rij te verwijderen waar een foreign key van een andere tabel naar verwijst.
Als voorbeeld gebruiken we een event waarvoor inschrijvingen bestaan.

Eerst een overzicht van de inschrijvingen:

In [16]:
SELECT lidnr, eventnr, maaltijd FROM inschrijvingen;

lidnr,eventnr,maaltijd
1,1,maaltijd A
1,2,maaltijd B
2,3,geen
3,4,maaltijd A
1,22,maaltijd A
2,22,maaltijd A


**Opdracht** Voer de volgende cel uit, voor het verwijderen van een event.

* Welke foutmelding krijg je?
* Ga na welke persoon niet gebruikt wordt in een inschrijving (via de cel daaronder).
* Verander de `x` in de daarop volgende cel door het nummer van een niet-gerefereerde event, en voer de cel uit.

In [18]:
DELETE 
FROM events
WHERE eventnr = 4;

**Opdracht**

* Pas de `x` in de cel hieronder aan zodat je een lid probeert te verwijderen waarvoor een inschrijving bestaat, en voer de cel uit;
* Welke foutmelding krijg je?
* Verander de `x` in het nummer van een lid zonder inschrijvingen, en voer de opdracht uit;
* Maak ter controle een overzicht van de leden.

In [21]:
DELETE FROM leden
 WHERE lidnr = 1 ;

In [22]:
DELETE 
FROM leden
WHERE lidnr = 4;

Het volgende voorbeeld is een SQL-demonstratie.

**Voorbeeld:** een SQL-query om de leden zonder inschrijvingen op te vragen.
Geef hiervan de voornaam, achternaam en email-adres.

In [25]:
SELECT voornaam, achternaam, email
FROM leden lid
WHERE lid.lidnr NOT IN
    (SELECT ins.lidnr FROM inschrijvingen ins);

voornaam,achternaam,email
